In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import keras

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split 

import

#### load data

In [3]:
df = pd.read_csv('data/binary_full.csv')
pd.set_option('display.max_columns', None)
df.drop('Unnamed: 0', axis=1, inplace=True)

#### retreive model

In [4]:
model = keras.models.load_model('saved_models/mlp_binary_1.h5')
model.compile(loss='binary_crossentropy', #categorical_hinge
              optimizer=SGD(lr=0.0005),
              metrics=['acc',tf.keras.metrics.AUC(), tf.keras.metrics.Recall()])

NameError: name 'SGD' is not defined

#### some functions to evaluate model predictions

In [ ]:
def confusion_eval(model, X_test, y_test, threshold=0.5)
    """
    Takes in model, testing data, and decision threshold and outputs prediction performance (confusion matrix,
    sensitivity, specificity, and accuracy)
    """
    y_predicted = (model.predict(X_test) >= threshold)

    conf_mat = confusion_matrix(Y_test, y_predicted)
    print(conf_mat)
    total = sum(sum(conf_mat))
    sensitivity = conf_mat[0, 0]/(conf_mat[0, 0] + conf_mat[1, 0])
    specificity = conf_mat[1, 1]/(conf_mat[1, 1] + conf_mat[0, 1])
    accuracy = (conf_mat[0, 0] + conf_mat[1, 1])/total

    print('specificity : ', specificity)
    print('sensitivity : ', sensitivity)
    print('accuracy : ', accuracy)
    return conf_mat, sensitivity, specificity, accuracy

### let's figure out the best threshold to minimize false negatives!

> Since this problem is a case of medical diagnostics, we care way more about correctly diagnosing the people who actually have CVDs so that they can get treatment, therefore we are willing to allow a few more false positive cases to ensure the best ultimate health outcomes for everyone.

In [ ]:
test_thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]

# specificities = []
# sensitivites = []
# accuracies = []

df_results = pd.DataFrame(columns = )
for threshold in thresholds:
    mat, spec, sens, acc = confusion_eval(model, X_test, y_test, threshold=threshold)
    
    specificities.append(spec)
    sensitivities.append(sens)
    accuracies.append(acc)

#### plot threshold results

In [ ]:
def plot_compare(metric):
    for thresh, result in zip(test_thresholds, metric):